In [22]:
from tensorflow.keras.layers import Input, LSTM, GRU, SimpleRNN, Dense, GlobalMaxPool1D,Reshape,Dropout,Lambda
# from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras import layers,models,Sequential
# from tensorflow.keras import layers, models

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import cross_val_score
from lightgbm import LGBMClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import KFold
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

In [66]:
files_path = 'CleanFiles/'
files_format = '.csv'
files_postfix = 'Extract'
# file_names = ['AgiaParaskevi','Aristotelous','Athens','Elefsina','Koropi','Liosia',
#               'Lykovrisi','Marousi','NeaSmirni','Patision','Peristeri',
#               'Pireus','Thrakomakedones']
file_names = ['Thrakomakedones']

In [24]:
def pm10_index(val):
    if val <= 25.0:
        return 0
    elif 26.0 <= val <= 50.0:
        return 1
    elif 51.0 <= val <= 90.0:
        return 2
    elif 91.0 <= val <= 180.0:
        return 3
    else:
        return 4

In [25]:
def get_time(tmp):
    return tmp[11:13]

In [26]:
def get_season(tmp):
    if tmp == 'Spring':
        return '1'
    elif tmp == 'Spring/Summer':
        return '2'
    elif tmp == 'Summer':
        return '3'
    elif tmp == 'Summer/Autumn':
        return '4'
    elif tmp == 'Autumn':
        return '5'
    elif tmp == 'Autumn/Winter':
        return '6'
    elif tmp == 'Winter/Spring':
        return '7'
    else:
        return '8'

In [27]:
def get_winddir(tmp):
    if tmp == 'N':
        return '1'
    elif tmp == 'NNE':
        return '2'
    elif tmp == 'NE':
        return '3'
    elif tmp == 'ENE':
        return '4'
    elif tmp == 'E':
        return '5'
    elif tmp == 'ESE':
        return '6'
    elif tmp == 'SE':
        return '7'
    elif tmp == 'SSE':
        return '8'
    elif tmp == 'S':
        return '9'
    elif tmp == 'SSW':
        return '10'
    elif tmp == 'SW':
        return '11'
    elif tmp == 'WSW':
        return '12'
    elif tmp == 'W':
        return '13'
    elif tmp == 'WNW':
        return '14'
    elif tmp == 'NW':
        return '15'
    else:
        return '16'

In [28]:
#neasmirni = pd.read_csv('CleanFiles/NeaSmirniExtract.csv', sep=',')
#neasmirni = neasmirni[['date_time','station_id','season','real_temp','real_windspd','real_winddir','real_humidity','forecast_tempC','forecast_windSpeed','forecast_windDirection','forecast_humidity','pm10']]
#neasmirni['pm10'] = neasmirni['pm10'].map(lambda a: pm10_index(a))
#temp = pd.read_csv('CleanFiles/PeristeriExtract.csv', sep=',')
#agiaparaskevi = pd.read_csv('CleanFiles/AgiaParaskeviExtract.csv', sep=',')

In [29]:
# T = 72
# D = 1
# X = []
# Y = []
# for t in range(len(neasmirni) - T):
#     x = neasmirni[t:t+T]
#     X.append(x)
#     y = neasmirni[t:t+T]
#     Y.append(y)

In [30]:
def create_xy(series, series2, window_size, prediction_horizon, shuffle = False):
    x = []
    y = []
    for i in range(0, len(series)):
        if len(series[(i + window_size):(i + window_size + prediction_horizon)]) < prediction_horizon:
            break
        x.append(np.array(series[i:(i + window_size)]))
        y.append(np.array(series2[(i + window_size):(i + window_size + prediction_horizon)]))
#         x.append(np.array(series[i:i+window_size][['station_id','season','real_temp','real_humidity','real_windspd','real_winddir','forecast_tempC','forecast_humidity','forecast_windSpeed','forecast_windDirection']]))
#         y.append(np.array(series[i+window_size:i+window_size+prediction_horizon]['pm10']))
    x = np.array(x)
    y = np.array(y)
    return x,y

In [31]:
parameters = {
    'n_estimators': 2000,
    'max_depth': 4,
    'num_leaves': 2**4,
    'learning_rate': 0.1,
    'boosting_type': 'dart'
}

In [234]:
def gru(units,drop,features, window, horizon):
    encoder_inputs = layers.Input(shape=(window,features))
    encoder = GRU(units, dropout=drop,return_state=True)
    _,encoder_states = encoder(encoder_inputs)
    decoder=layers.RepeatVector(horizon)(encoder_states)
    out = layers.TimeDistributed(Dense(1))(decoder)
    model = models.Model(encoder_inputs, out)
    model.compile(loss='mse', optimizer=RMSprop())
    return model

In [235]:
def seq2seq(window, horizon, units,bn,drop,feautures,channel=1):
    encoder_inputs = layers.Input(shape=(window,feautures))

    encoder = GRU(units, dropout=drop,return_state=True)
    _,encoder_states = encoder(encoder_inputs)
    if bn:
        encoder_states=layers.BatchNormalization()(encoder_states)
    decoder=layers.RepeatVector(horizon)(encoder_states)
    decoder_gru = GRU(units, dropout=drop, return_sequences=True, return_state=False)
    decoder = decoder_gru(decoder, initial_state=encoder_states)
    
    out = layers.TimeDistributed(Dense(channel))(decoder)
    model = models.Model(encoder_inputs, out)
    model.compile(loss='mse', optimizer=RMSprop(),metrics=['accuracy'])
    return model

In [230]:
def gatedDNN(features, horizon):
#     model = Sequential()
#     model.add(layers.Input(shape=(72,features)))
#     model.add(layers.Dense(2, activation='relu'))
#     model.compile(loss='mean_squared_error', optimizer='adam')
#     return model

    model = Sequential([
    Lambda(lambda x: x[:, -1:, :]),
    Dense(50, activation='relu'),
    Dense(horizon*1),
    Reshape([horizon, 1])
    ])
    model.compile(loss='mse', optimizer=RMSprop(),metrics=['accuracy'])
    return model

In [224]:
def lstm(features, window, horizon):
    model = Sequential([
    LSTM(10, input_shape=(window, features), return_sequences=True),
    Dropout(0.5),
    LSTM(10, return_sequences=True),
    Dense(horizon),
    Reshape([horizon, 1])
    ])
    
    model.compile(optimizer=RMSprop(), loss='mean_squared_error', metrics=['accuracy'])
    return model

In [275]:
def loadToModel(files, formating, pathFiles, postfx, params, predicts):
    T = 72
    D = 1
    X = []
    Y = []
    for fileName in files:
        window = 72 
        horizon = 48
        accuracy = 0
        #df = pd.read_csv(pathFiles + fileName + postfx + formating, sep=',')
        df = pd.read_csv(pathFiles + fileName + formating, sep=',')
        
        #df['pm10'] = df['pm10'].map(lambda a: pm10_index(a))
        #df['pm10'] = df['pm10'].astype(int)
        
        df['date_time'] = df['date_time'].astype("|S")
        df['season'] = df['season'].astype("|S")
        
        #df['forecast_windDirection'] = df['forecast_windDirection'].astype("|S")
        #df['real_winddir'] = df['real_winddir'].astype("|S")
        df['date_time'] = df['date_time'].map(lambda a: get_time(a))
        df['date_time'] = df['date_time'].astype(np.float64)
        df['season'] = df['season'].map(lambda a: get_season(a))
        df['season'] = df['season'].astype(np.float64)
        
        #df['forecast_windDirection'] = df['forecast_windDirection'].map(lambda a: get_winddir(a))
        
        df['forecast_windDirection'] = df['forecast_windDirection'].astype(np.float64)
        
        #df['real_winddir'] = df['real_winddir'].map(lambda a: get_winddir(a))
        #df['real_winddir'] = df['real_winddir'].astype(np.float64)
        #df['pm10'] = df['pm10'].astype('int')
        
        df['station_id'] = df['station_id'].astype(int)
#         df['forecast_tempC'] = df['forecast_tempC'].astype(np.float64)
#         df['forecast_humidity'] = df['forecast_humidity'].astype(np.float64)
#         df['forecast_windSpeed'] = df['forecast_windSpeed'].astype(np.float64)

        scaler = MinMaxScaler(feature_range=(0, 1))
        names = ['forecast_tempC','forecast_humidity','forecast_windSpeed','forecast_windDirection','pm10']
        d = scaler.fit_transform(df)
        df = pd.DataFrame(df, columns=names)
        

        X,y = create_xy(df[['forecast_tempC','forecast_humidity','forecast_windSpeed','forecast_windDirection','pm10']],df[['pm10']],window, horizon)
        accuracy = 0.0
        print(fileName + ' begins prediction')
#        model = GradientBoostingRegressor(learning_rate=0.05,max_features=0.6,max_leaf_nodes=31,n_estimators=200)
#         model = seq2seq(72,48,5,False,0.5,5,1)
#         model = lstm(5)
        model = gatedDNN(5, 48)
#        model.compile(optimizer=RMSprop(), loss='mse')
        
#         model = MultiOutputRegressor(gbm,-1)
#         model = lgb.LGBMRegressor()
#         model = XGBClassifier()
#         model = XGBClassifier(gamma=0.1, max_depth=50, n_estimators=80, learning_rate=0.1, colsample_bytree=.7, reg_alpha=4, objective='binary:logistic', eta=0.3, silent=1, subsample=0.8)
        
        kf = KFold(n_splits=5)

#         nsamples, nx, ny = np.array(X).shape
#         X = X.reshape((nsamples,nx*ny))
#         nsamples2, nx2 = np.array(y).shape
#         y = y.reshape((nsamples2,nx2))

        print(X.shape)
        print(y.shape)
        
#         epochs = 2
#         batch_size = 32

        for trainI,testI in kf.split(X):
            model.fit(X[trainI],y[trainI])
            predictions = model.predict(X[testI])
            predicts.append(predictions)
#             print("*************")
#             print(y[testI])
#            accuracy += accuracy_score(y[testI], predictions)
        
#         _, test_acc = model.evaluate(X,y,verbose=0)
#         accuracy = test_acc
#         print('Accuracy ' + fileName + ': %.3f' % (accuracy/5))

In [276]:
predicts = []

In [277]:
loadToModel(file_names, files_format, files_path, files_postfix, parameters,predicts)

Thrakomakedones begins prediction
(43729, 72, 5)
(43729, 48, 1)
Train on 34983 samples
34983/34983 [==============================] - 4s 103us/sample - loss: 203.8610 - acc: 0.0154
Train on 34983 samples
34983/34983 [==============================] - 3s 78us/sample - loss: 279.8574 - acc: 0.0125
Train on 34983 samples
34983/34983 [==============================] - 3s 79us/sample - loss: 217.9183 - acc: 0.0148
Train on 34983 samples
34983/34983 [==============================] - 3s 81us/sample - loss: 251.3835 - acc: 0.0140
Train on 34984 samples
34984/34984 [==============================] - 3s 78us/sample - loss: 272.8770 - acc: 0.0120


In [ ]:
df = pd.read_csv('CleanFiles/AgiaParaskevi.csv', sep=',')

In [49]:
df

,Unnamed: 0,date_time,station_id,forecast_tempC,forecast_humidity,season,forecast_windDirection,forecast_windSpeed,pm10,pm25,o3,no,no2
0,0,2016-01-01 00:00:00,2,3,69,Winter,0.0,23.0,27.0,22.0,-9999.0,1.0,12.0
1,1,2016-01-01 01:00:00,2,3,69,Winter,119.0,22.0,30.0,29.0,-9999.0,1.0,16.0
2,2,2016-01-01 02:00:00,2,3,69,Winter,238.0,20.0,30.0,25.0,-9999.0,1.0,7.0
3,3,2016-01-01 03:00:00,2,3,69,Winter,357.0,19.0,8.0,8.0,-9999.0,1.0,8.0
4,4,2016-01-01 04:00:00,2,3,70,Winter,355.0,17.0,9.0,10.0,-9999.0,1.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43843,43843,2020-12-31 19:00:00,2,13,73,Winter,215.0,21.0,30.0,12.0,47.0,1.0,5.0
43844,43844,2020-12-31 20:00:00,2,13,74,Winter,222.0,17.0,30.0,16.0,40.0,1.0,5.0
43845,43845,2020-12-31 21:00:00,2,12,74,Winter,228.0,14.0,30.0,21.0,48.0,1.0,5.0
43846,43846,2020-12-31 22:00:00,2,12,74,Winter,240.0,12.0,30.0,14.0,50.0,1.0,5.0
